## The forward and backward passes

In [ ]:
import pickle,gzip,math,os,time,shutil,torch,matplotlib as mpl, numpy as np
from pathlib import Path
from torch import tensor
from fastcore.test import test_close
torch.manual_seed(42)

mpl.rcParams['image.cmap'] = 'gray'
torch.set_printoptions(precision=2, linewidth=125, sci_mode=False)
np.set_printoptions(precision=2, linewidth=125)

path_data = Path('data')
path_gz = path_data/'mnist.pkl.gz'
with gzip.open(path_gz, 'rb') as f: 
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
x_train, y_train, x_valid, y_valid = map(tensor, [x_train, y_train, x_valid, y_valid])

## Foundations version

### Basic architecture

In [ ]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

Build a simple model with three layers: 
- L1: hidden linear layer with `nh` units
- L2: relu layer
- O: output linear layer with ` unit

In [ ]:
# num hidden units
nh = 50

Initialize the two sets of weight and biases matices

$$
Y = X \times W + B \\ \updownarrow \\
\begin{bmatrix}
  y_{0} & y_{1} & \dots & y_{50}\\
\end{bmatrix} = 
\begin{bmatrix}
  x_{0} & x_{1} & \dots & x_{783}\\
\end{bmatrix} \times
\begin{bmatrix}
  w_{0,0}    & w_{0,1}    & \dots & w_{0, 49} \\
  w_{1, 0}   & w_{1, 1}   & \dots & w_{1, 49} \\
             &            &\vdots             \\
  w_{783, 0} & w_{783, 1} & \dots & w_{783, 49}
\end{bmatrix} +
\begin{bmatrix}
  b_{0} & b_{1} & \dots & b_{50}\\
\end{bmatrix}
$$

In [ ]:
w1 = torch.randn(m,nh)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)
b2 = torch.zeros(1)

w1.shape, w2.shape

(torch.Size([784, 50]), torch.Size([50, 1]))

Create the layers and model

In [ ]:
def lin(x, w, b): 
    """Linear layer"""
    return x@w + b

In [ ]:
t = lin(x_valid, w1, b1)
t.shape

torch.Size([10000, 50])

In [ ]:
def relu(x): 
    """Rectified Linear Unit"""
    return x.clamp_min(0.)

In [ ]:
t = relu(t)
t.shape

torch.Size([10000, 50])

In [ ]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    o = lin(l2, w2, b2)
    return o

In [ ]:
res = model(x_valid)
res.shape

torch.Size([10000, 1])

### Loss function: MSE

(Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [ ]:
res.shape,y_valid.shape

(torch.Size([10000, 1]), torch.Size([10000]))

If we just take the difference between `res` and `y_valid`, we will broadcast in the wrong way:
- trailing dimension: 1 in `res` `[10000,1]` compared to 10000 `y_valid` `[10000]` => `res` is broadcasted to `[10000,10000]`
- other dimension: 10000 in `res` `[10000,1]` compared to None in `y_valid` `[10000]` => `y_valid`  is broadcasted to `[10000,10000]`

In [ ]:
(res-y_valid).shape

torch.Size([10000, 10000])

We need to get rid of that trailing (,1), in order to use `mse`.

In [ ]:
res[:,0].shape

torch.Size([10000])

Another way of doing this is to use `squeeze`; `squeeze` will remove an empty dimension (1 in shape on that axis)

In [ ]:
res.squeeze().shape

torch.Size([10000])

In [ ]:
(res[:,0]-y_valid).shape, (res.squeeze()-y_valid).shape

(torch.Size([10000]), torch.Size([10000]))

Also, `y_train` and `y_valid` have a dtype `int64` while `res` or `preds` are `float32`. It is a good practice to align both

In [ ]:
y_train.dtype, res.dtype

(torch.int64, torch.float32)

In [ ]:
y_train,y_valid = y_train.float(),y_valid.float()

preds = model(x_train)
preds.shape, preds.dtype, y_train.dtype

(torch.Size([50000, 1]), torch.float32, torch.float32)

In [ ]:
def mse(output, targ): 
    return (output.squeeze()-targ).pow(2).mean()

In [ ]:
mse(preds, y_train)

tensor(4308.76)

### Gradients and backward pass

Symbolic derivation !

In [ ]:
from sympy import symbols,diff
x,y = symbols('x y')
diff(x**2, x)

2*x

In [ ]:
diff(3*x**2+9, x)

6*x

In [ ]:
# def lin_grad(inp, out, w, b):
#     """grad of matmul with respect to input"""
#     inp.g = out.g @ w.t()
#     w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
#     b.g = out.g.sum(0)

Steps to build the function for gradient

1. Use python debugger to understand what is going on
1. Lots of unsqueeze, ... means we probably can use a einsum. check with the debugger
1. Replace by a einsum
1. Actually, looks like a matmul. check and replace

**Backward Pass**:

<img src="images/backprop.png" width="50%">

Also see this [link](https://nasheqlbrm.github.io/blog/posts/2021-11-13-backward-pass.html) for a post with both math and code.

In [ ]:
import pdb

def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
#     pdb.set_trace()
    inp.g = out.g @ w.t()
#     i,o = inp.unsqueeze(-1), out.g.unsqueeze(1)
#     w.g = (i * o).sum(0)
    w.g = inp.T@out.g
    b.g = out.g.sum(0)

In [ ]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2

    diff = out[:,0] - targ
    loss = diff.pow(2).mean()
    
#     pdb.set_trace()
    
    # backward pass:
    # 1. gradient loss w.r.t out (dLoss/dout) is added as an attribute to the tensor out
    out.g = 2. * diff[:,None] / inp.shape[0]

    # 2. gradient out (L2 output) w.r.t l2 (L2 input)
    lin_grad(l2, out, w2, b2)
    
    # 3. gradient l2 (Relu output) w.r.t. l1 (L1 output)
    l1.g = (l1 > 0).float() * l2.g
    
    # 4. gradient l1 (L1 output) w.r.t. inputs
    lin_grad(inp, l1, w1, b1)

In [ ]:
forward_and_backward(x_train, y_train)

#### Debugger `pdb`

**Past tracing inside the two functions**
```python
> /tmp/ipykernel_12530/3466932735.py(14)forward_and_backward()
     12     # backward pass:
     13     # 1. gradient loss w.r.t out (dLoss/dout) is added as an attribute to the tensor out
---> 14     out.g = 2. * diff[:,None] / inp.shape[0]
     15 
     16     # 2. gradient out (L2 output) w.r.t l2 (L2 input)

ipdb> p out.g
*** AttributeError: 'Tensor' object has no attribute 'g'
ipdb> n
> /tmp/ipykernel_12530/3466932735.py(17)forward_and_backward()
     15 
     16     # 2. gradient out (L2 output) w.r.t l2 (L2 input)
---> 17     lin_grad(l2, out, w2, b2)
     18 
     19     # 3. gradient l2 (Relu output) w.r.t. l1 (L1 output)

ipdb> p out.g
tensor([[-16.68],
        [  9.42],
        [-18.07],
        [  6.24]])
ipdb> c
> /tmp/ipykernel_12530/1343070491.py(6)lin_grad()
      4     # grad of matmul with respect to input
      5     pdb.set_trace()
----> 6     inp.g = out.g @ w.t()
      7     i,o = inp.unsqueeze(-1), out.g.unsqueeze(1)
      8     w.g = (i * o).sum(0)

ipdb> p inp.g
*** AttributeError: 'Tensor' object has no attribute 'g'
ipdb> n
> /tmp/ipykernel_12530/1343070491.py(7)lin_grad()
      5     pdb.set_trace()
      6     inp.g = out.g @ w.t()
----> 7     i,o = inp.unsqueeze(-1), out.g.unsqueeze(1)
      8     w.g = (i * o).sum(0)
      9     w.g = inp.T@out.g

ipdb> p inp.g.shape
torch.Size([4, 50])
ipdb> p out.shape
torch.Size([4, 1])
ipdb> p out.g.shape
torch.Size([4, 1])
ipdb> p inp.unsqueeze(-1).shape
torch.Size([4, 50, 1])
ipdb> p out.g.unsqueeze(-1)
tensor([[[-16.68]],

        [[  9.42]],

        [[-18.07]],

        [[  6.24]]])
ipdb> p out.g.unsqueeze(-1).shape
torch.Size([4, 1, 1])
ipdb> n
> /tmp/ipykernel_12530/1343070491.py(8)lin_grad()
      6     inp.g = out.g @ w.t()
      7     i,o = inp.unsqueeze(-1), out.g.unsqueeze(1)
----> 8     w.g = (i * o).sum(0)
      9     w.g = inp.T@out.g
     10     b.g = out.g.sum(0)

ipdb> p i.shape,o.shape
(torch.Size([4, 50, 1]), torch.Size([4, 1, 1]))
ipdb> (i*o).shape
torch.Size([4, 50, 1])
ipdb> (inp.T@out.g).shape
torch.Size([50, 1])
ipdb> (i * o).sum(0).shape
torch.Size([50, 1])
ipdb> q
```

```python
ipdb> h

Documented commands (type help <topic>):
========================================
EOF    commands   enable    ll        pp       s                until 
a      condition  exit      longlist  psource  skip_hidden      up    
alias  cont       h         n         q        skip_predicates  w     
args   context    help      next      quit     source           whatis
b      continue   ignore    p         r        step             where 
break  d          interact  pdef      restart  tbreak         
bt     debug      j         pdoc      return   u              
c      disable    jump      pfile     retval   unalias        
cl     display    l         pinfo     run      undisplay      
clear  down       list      pinfo2    rv       unt            

Miscellaneous help topics:
==========================
exec  pdb

ipdb> p
*** SyntaxError: invalid syntax
ipdb> p inp.shape
torch.Size([50000, 50])
ipdb> print(inp.shape)
torch.Size([50000, 50])
ipdb> p out.g.shape
torch.Size([50000, 1])
ipdb> p (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0).shape
torch.Size([50, 1])
ipdb> p (inp.unsqueeze(-1) * out.g.unsqueeze(1)).shape
torch.Size([50000, 50, 1])
```

#### end debugger

In [ ]:
print(w1.shape, w1.g.shape)
print(b1.shape, b1.g.shape)
print(w2.shape, w2.g.shape)
print(b2.shape, b2.g.shape)

torch.Size([784, 50]) torch.Size([784, 50])
torch.Size([50]) torch.Size([50])
torch.Size([50, 1]) torch.Size([50, 1])
torch.Size([1]) torch.Size([1])


In [ ]:
# Save for testing against later
def get_grad(x): return x.g.clone()
chks = w1,w2,b1,b2,x_train
grads = w1g,w2g,b1g,b2g,ig = tuple(map(get_grad, chks))

We cheat a little bit and use PyTorch autograd to check our results.
- we create a set of tensors with autograd w12,w22,b12,b22,xt2 with same values as w1,w2,b1,b2,x_train
- we make a forward pass, and pytorch calculates the grads automatically
- we compare the two sets of gradients

In [ ]:
def mkgrad(x): return x.clone().requires_grad_(True)
ptgrads = w12,w22,b12,b22,xt2 = tuple(map(mkgrad, chks))

In [ ]:
def forward_and_loss(inp, targ):
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [ ]:
loss = forward_and_loss(xt2, y_train)
loss.backward()

In [ ]:
from fastcore.test import test_close

In [ ]:
test_close(w22.grad, w2g, eps=0.01)
test_close(b22.grad, b2g, eps=0.01)
test_close(w12.grad, w1g, eps=0.01)
test_close(b12.grad, b1g, eps=0.01)
test_close(xt2.grad, ig , eps=0.01)

In [ ]:
for a,b in zip(grads, ptgrads): 
    test_close(a, b.grad, eps=0.01)

## Refactor model

Code is rather clunky above. We will refactor
1. layer as classes, 
    - with a `__call__` method to make them callable. 
    - `__call__` will not only calculate the output but also will store input and outputs for the backward pass. this avoids to compute all outputs twice, at the expense of memory
    - with a backward method for backward pass

### Layers as classes

In [ ]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): 
        self.inp.g = (self.inp > 0).float() * self.out.g

# self.out.g does not exist at the time of forward pass, but will be set by the backward pass of the following layer

In [ ]:
class Lin():
    # __init__ because we need to initialize the weights upon layer creation
    def __init__(self, w, b): self.w, self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out

    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = self.inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [ ]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: 
            x = l(x)
        self.preds = x
        self.final_loss = self.loss(x, targ)
        return self.final_loss
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): 
            l.backward()

> In this model, the loss function is in the model (used often for huggingface models).
>
>In fastai, it is often outside and calculate separetely.
>
>Both are fine

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
loss = model(x_train, y_train)
loss

tensor(4308.76)

In [ ]:
model.preds.shape, model.final_loss

(torch.Size([50000, 1]), tensor(4308.76))

In [ ]:
model.layers[0].w.shape, model.layers[0].w.g.shape, model.layers[0].w.g.max(), model.layers[0].w.g.min()

(torch.Size([784, 50]), torch.Size([784, 50]), tensor(151.95), tensor(-70.73))

In [ ]:
model.backward()

In [ ]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

It is better, but still have repeated code, e.g. the `__call__` method in each layer. 

Let's have a template as `Module`:

### Module.forward()

Structure of `Module`
- `__call__` template method for all modules, like the one in each layer, but generalized to takes any list of positional args (inputs), save them, run a forward pass, and save the results
- `forward` depends on each specific layer
- `backward` is the same for all module, but calls a specific method `bwd`

In [ ]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out

    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)
    def bwd(self): raise Exception('not implemented')

Simplifies the layers. For instance, previous Relu was:
```python
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)
        return self.out
    
    def backward(self): 
        self.inp.g = (self.inp>0).float() * self.out.g
```

In [ ]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)
    def bwd(self, out, inp): 
        inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
    def forward(self, inp): 
        return inp@self.w + self.b
    def bwd(self, out, inp):
        inp.g = self.out.g @ self.w.t()
        self.w.g = inp.t() @ self.out.g
        self.b.g = self.out.g.sum(0)

In [ ]:
class Mse(Module):
    def forward (self, inp, targ): 
        self.diff = (inp.squeeze() - targ)
        return self.diff.pow(2).mean()
        
    def bwd(self, out, inp, targ): 
        inp.g = 2*(self.diff).unsqueeze(-1) / targ.shape[0]

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
loss = model(x_train, y_train)

In [ ]:
model.backward()

In [ ]:
test_close(w2g, w2.g, eps=0.01)
test_close(b2g, b2.g, eps=0.01)
test_close(w1g, w1.g, eps=0.01)
test_close(b1g, b1.g, eps=0.01)
test_close(ig, x_train.g, eps=0.01)

### Autograd

Now we may use the pytorch package. And pytorch calculates the gradient itself, no need to define `backward`

In [ ]:
from torch import nn
import torch.nn.functional as F

In [ ]:
class Linear(nn.Module):
    def __init__(self, n_in, n_out):
        super().__init__()
        self.w = torch.randn(n_in,n_out).requires_grad_()
        self.b = torch.zeros(n_out).requires_grad_()
    def forward(self, inp): return inp@self.w + self.b

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [Linear(n_in,nh), nn.ReLU(), Linear(nh,n_out)]
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return F.mse_loss(x, targ[:,None])

In [ ]:
model = Model(m, nh, 1)
loss = model(x_train, y_train)
loss.backward()

In [ ]:
l0 = model.layers[0]
l0.b.grad

tensor([-19.60,  -2.40,  -0.12,   1.99,  12.78, -15.32, -18.45,   0.35,   3.75,  14.67,  10.81,  12.20,  -2.95, -28.33,
          0.76,  69.15, -21.86,  49.78,  -7.08,   1.45,  25.20,  11.27, -18.15, -13.13, -17.69, -10.42,  -0.13, -18.89,
        -34.81,  -0.84,  40.89,   4.45,  62.35,  31.70,  55.15,  45.13,   3.25,  12.75,  12.45,  -1.41,   4.55,  -6.02,
        -62.51,  -1.89,  -1.41,   7.00,   0.49,  18.72,  -4.84,  -6.52])